# Encoder Decoder 

### Imports

In [1]:
import pandas as pd
import numpy as np
import re
import string
from sklearn.utils import shuffle

### Importing Data

In [2]:
df = pd.read_csv('ASL_English.csv')
df.head()

,English,ASL Gloss
0,Do you want a ride to the mall?,M-A-L-L RIDE WANT YOU Q
1,Yes I want to start buying Christmas gifts soon,YES SOON CHRISTMAS GIFTS START BUYING WANT ME
2,Please sit in this chair,THIS CHAIR PLEASE SIT
3,I like to fly small planes,SMALL PLANES FLY LIKE ME
4,He will go later,HE GO WILL


In [3]:
df.shape

(1670, 2)

### Preprocessing

Note:
- Replace the numbers/digits
- Check regarding Finger spellings
- Check if it is required to add start and end tokens to target sequences

In [4]:
replacements = {'1': "one ", '2':"two ", '3':"three ", '4':"four ", '5':"five ", '6':"six ", '7':"seven ", '8':"eight ", '9':"nine ", '0':"zero "}
df['English'] = df['English'].apply(lambda x: re.sub('(\d)', lambda m: replacements[m.group()], x))
df['ASL Gloss'] = df['ASL Gloss'].apply(lambda x: re.sub('(\d)', lambda m: replacements[m.group()], x))

# Remove extra spaces
df['English'] = df['English'].apply(lambda x: x.strip())
df['ASL Gloss'] = df['ASL Gloss'].apply(lambda x: x.strip())

# Lowercase all characters
df['English'] = df['English'].apply(lambda x: x.lower())
df['ASL Gloss'] = df['ASL Gloss'].apply (lambda x: x.lower())

# Removing double spaces
df['English'] = df['English'].apply(lambda x: x.replace('  ', ' '))
df['ASL Gloss'] = df['ASL Gloss'].apply(lambda x: x.replace('  ', ' '))

# Remove quotes # Might not need this
# df['English'] = df['English'].apply(lambda x: re.sub (r"'", '', x))
# df['ASL Gloss'] = df['ASL Gloss'].apply(lambda x: re.sub (r"'", '', x))

# Remove all special character
df['English'] = df['English'].apply(lambda x: ''.join (ch for ch in x if ch not in set(string.punctuation)))
df['ASL Gloss'] = df['ASL Gloss'].apply(lambda x: ''.join (ch for ch in x if ch not in set(string.punctuation)))

# Check if dataset has numbers
#print(df['English'].str.contains(r'\d').any())
#print(df['ASL Gloss'].str.contains(r'\d').any())

# Add tokens to target sequence
df['English'] = df['English'].apply(lambda x : 'START_ ' + x + ' _END')

In [5]:
df.head(10)

,English,ASL Gloss
0,START_ do you want a ride to the mall _END,mall ride want you q
1,START_ yes i want to start buying christmas gi...,yes soon christmas gifts start buying want me
2,START_ please sit in this chair _END,this chair please sit
3,START_ i like to fly small planes _END,small planes fly like me
4,START_ he will go later _END,he go will
5,START_ mike is walking over to my house _END,my house mike walk will
6,START_ i havent eaten _END,me eat not yetstructuring sentences
7,START_ he ran _END,he ran
8,START_ he ran _END,ran him
9,START_ she fell _END,she fell


In [6]:
# df['ASL Gloss'].str.len().sort_values(ascending=False).head()
# df['English'].str.len().sort_values(ascending=False).head()

In [7]:
# Vocabulary of English
all_eng_words = set()
for eng in df['English']:
    for word in eng.split():
        if word not in all_eng_words:
            all_eng_words.add(word)

# Vocabulary of ASL 
all_ASL_words = set()
for asl in df['ASL Gloss']:
    for word in asl.split():
        if word not in all_ASL_words:
            all_ASL_words.add(word)

In [8]:
# Max Length of source sequence
lenght_list=[]
for l in df ['English']:
    lenght_list.append(len(l.split(' ')))
max_length_tar = np.max(lenght_list)
print("Max length target: ", max_length_tar)

Max length target:  25


In [9]:
# Max Length of target sequence
lenght_list=[]
for l in df ['ASL Gloss']:
    lenght_list.append(len(l.split(' ')))
max_length_src = np.max(lenght_list)
print("Max length sorce: ", max_length_src)

Max length sorce:  18


In [10]:
input_words = sorted(list(all_ASL_words))
target_words = sorted(list(all_eng_words))

# Calculate Vocab size for both source and target
num_encoder_tokens = len(all_ASL_words) + 1
num_decoder_tokens = len(all_eng_words) + 1

num_encoder_tokens, num_decoder_tokens

(1948, 2002)

In [11]:
num_decoder_tokens += 1 # For zero padding
num_decoder_tokens

2003

In [12]:
# Create word to token dictionary for both source and target
input_token_index = dict([(word, i+1) for i, word in enumerate(input_words)])
target_token_index = dict([(word, i+1) for i, word in enumerate(target_words)])

# Create token to word dictionary for both source and target
reverse_input_char_index = dict((i, word) for word, i in input_token_index.items())
reverse_target_char_index = dict((i, word) for word, i in target_token_index.items())

In [13]:
shuffle(df).head(10)

,English,ASL Gloss
1387,START_ labor day is monday september one st _END,labor day when monday september one st
986,START_ i like trees _END,i like trees
1423,START_ today the weather is snowy _END,weather today what snow
1156,START_ bill was born in one nine six four _END,year one nine six four bill born
1168,START_ emily was born in the month of march _END,emily born month march
964,START_ there is no way im buying that bag it c...,that bag no i not buy cost an arm and a leg
1391,START_ hanukkah is sunday december two one _END,hanukkah when sunday december two one
1597,START_ my aunt was married in alabama _END,my aunt married where alabama
1617,START_ my aunt lives in maine _END,my aunt live where maine
1087,START_ do you love playing the piano _END,you love piano


Make a 90–10 train and test split and write a Python generator function to load the data in batches as follows:

In [14]:
from sklearn.model_selection import train_test_split
from keras.layers import Input, LSTM, Embedding, Dense
from keras.models import Model
import tensorflow as tf

In [15]:
# Train - Test Split
X, y = df['ASL Gloss'], df['English']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.1)
X_train.shape, X_test.shape

((1503,), (167,))

Save the train and test dataframes for reproducing the results later, as they are shuffled.

In [16]:
X_train.to_pickle('Weights_ASL/X_train.pkl')
X_test.to_pickle('Weights_ASL/X_test.pkl')

In [17]:
def generate_batch(X = X_train, y = y_train, batch_size = 128):
    ''' Generate a batch of data '''
    while True:
        for j in range(0, len(X), batch_size):
            encoder_input_data = np.zeros((batch_size, max_length_src),dtype='float32')
            decoder_input_data = np.zeros((batch_size, max_length_tar),dtype='float32')
            decoder_target_data = np.zeros((batch_size, max_length_tar, num_decoder_tokens),dtype='float32')
            for i, (input_text, target_text) in enumerate(zip(X.iloc[j:j+batch_size], y.iloc[j:j+batch_size])):
                for t, word in enumerate(input_text.split()):
                    encoder_input_data[i, t] = input_token_index[word] # encoder input seq
                for t, word in enumerate(target_text.split()):
                    if t<len(target_text.split())-1:
                        decoder_input_data[i, t] = target_token_index[word] # decoder input seq
                    if t>0:
                        # decoder target sequence (one hot encoded)
                        # does not include the START_ token
                        # Offset by one timestep
                        decoder_target_data[i, t - 1, target_token_index[word]] = 1.
            yield([encoder_input_data, decoder_input_data], decoder_target_data)

Encoder - Decoder Model Architecture

In [18]:
latent_dim = 50

In [19]:
# Encoder
encoder_inputs = Input(shape=(None,))
enc_emb =  Embedding(num_encoder_tokens, latent_dim, mask_zero = True)(encoder_inputs)
encoder_lstm = LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder_lstm(enc_emb)

# We discard `encoder_outputs` and only keep the states.
encoder_states = [state_h, state_c]

In [20]:
# Set up the decoder, using `encoder_states` as initial state.
decoder_inputs = Input(shape=(None,))
dec_emb_layer = Embedding(num_decoder_tokens, latent_dim, mask_zero = True)
dec_emb = dec_emb_layer(decoder_inputs)

# We set up our decoder to return full output sequences,
# and to return internal states as well. We don't use the
# return states in the training model, but we will use them in inference.
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(dec_emb, initial_state=encoder_states)

# Use a softmax to generate a probability distribution over the target vocabulary for each time step
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

# Define the model that will turn
# `encoder_input_data` & `decoder_input_data` into `decoder_target_data`
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

In [21]:
# Compile the model
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['acc'])

We train the network for 50 epochs with a batch size of 128.

In [22]:
train_samples = len(X_train)
val_samples = len(X_test)
batch_size = 128
epochs = 50
# epochs = 10

In [23]:
model.fit(generate_batch(X_test, y_test, batch_size = batch_size),
            batch_size = batch_size,
            steps_per_epoch = train_samples//batch_size,
            epochs=epochs,
            validation_data = generate_batch(X_test, y_test, batch_size = batch_size),
            validation_steps = val_samples//batch_size)

Epoch 1/50
11/11 [==============================] - 24s 1s/step - loss: 7.5888 - acc: 0.1077 - val_loss: 7.5743 - val_acc: 0.1338
Epoch 2/50
11/11 [==============================] - 4s 336ms/step - loss: 7.5543 - acc: 0.1316 - val_loss: 7.5255 - val_acc: 0.1338
Epoch 3/50
11/11 [==============================] - 3s 320ms/step - loss: 7.4333 - acc: 0.1322 - val_loss: 7.2704 - val_acc: 0.1338
Epoch 4/50
11/11 [==============================] - 4s 338ms/step - loss: 6.9586 - acc: 0.1316 - val_loss: 6.6637 - val_acc: 0.1338
Epoch 5/50
11/11 [==============================] - 4s 313ms/step - loss: 6.2809 - acc: 0.1322 - val_loss: 6.0776 - val_acc: 0.1338
Epoch 6/50
11/11 [==============================] - 3s 314ms/step - loss: 5.7294 - acc: 0.1316 - val_loss: 5.7594 - val_acc: 0.1338
Epoch 7/50
11/11 [==============================] - 4s 359ms/step - loss: 5.4772 - acc: 0.1322 - val_loss: 5.5804 - val_acc: 0.1338
Epoch 8/50
11/11 [==============================] - 4s 332ms/step - loss: 5.28

Always remember to save the weights

In [30]:
model.save_weights('Weights_ASL/nmt_weights.h5')

Load the weights, if you close the application

In [31]:
model.load_weights('Weights_ASL/nmt_weights.h5')

Inference Setup

In [32]:
# Encode the input sequence to get the "thought vectors"
encoder_model = Model(encoder_inputs, encoder_states)

# Decoder setup
# Below tensors will hold the states of the previous time step
decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

dec_emb2= dec_emb_layer(decoder_inputs) # Get the embeddings of the decoder sequence

# To predict the next word in the sequence, set the initial states to the states from the previous time step
decoder_outputs2, state_h2, state_c2 = decoder_lstm(dec_emb2, initial_state=decoder_states_inputs)
decoder_states2 = [state_h2, state_c2]
decoder_outputs2 = decoder_dense(decoder_outputs2) # A dense softmax layer to generate prob dist. over the target vocabulary

# Final decoder model
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs2] + decoder_states2)

Finally, we generate the output sequence by invoking the above setup in a loop as follows

Decode sample sequeces

In [33]:
def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)
    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1,1))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0] = target_token_index['START_']

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += ' '+sampled_char

        # Exit condition: either hit max length
        # or find stop character.
        if (sampled_char == '_END' or
           len(decoded_sentence) > 50):
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1,1))
        target_seq[0, 0] = sampled_token_index

        # Update states
        states_value = [h, c]

    return decoded_sentence

Evaluation on Train Dataset

In [34]:
train_gen = generate_batch(X_train, y_train, batch_size = 1)
k=-1

In [63]:
k+=1
(input_seq, actual_output), _ = next(train_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input ASL sentence:', X_train.iloc[k:k+1].values[0])
print('Actual English Translation:', y_train.iloc[k:k+1].values[0][6:-4])
print('Predicted English Translation:', decoded_sentence[:-4])

1/1 [==============================] - 0s 36ms/step
Input ASL sentence: my favorite kind popcorn which kettle corn
Actual English Translation:  my favorite kind of popcorn is kettle corn 
Predicted English Translation:  my my is is 
